In [ ]:
import numpy as np
import scipy.optimize as opt

n_vec = np.array([1.0, 1.0, 0.2])
r_path = np.array([0.04, 0.05, 0.051])
w_path = np.array([1.2, 1.18, 1.17])
beta = 0.9
sigma = 2.2
b_2tpl = 1.0


def Eul_err2(b3, *args):
    npath, rpath,  wpath, beta, sigma, b_2tpl = args
    c2 = (1 + rpath[0]) * b_2tpl + wpath[0] * npath[0] - b3
    c3 = (1 + rpath[1]) * b3 + wpath[1] * npath[1]
    MU2 = c2 ** (-sigma)
    MU3 = c3 ** (-sigma)
    eul_err = MU2 - beta * (1 + rpath[1]) * MU3
    
    return eul_err

def Eul_err1(b2, *args):
    npath, rpath,  wpath, beta, sigma, b3 = args
    c1 = wpath[0] * npath[0] - b2
    c2 = (1 + rpath[1]) * b2 + wpath[1] * npath[1] - b3
    MU1 = c1 ** (-sigma)
    MU2 = c2 ** (-sigma)
    eul_err = MU1 - beta * (1 + rpath[1]) * MU2
    return eul_err

b3_init = 0.1
b2_init = 0.1

b3_args = (n_vec[1:], r_path[1:], w_path[1:],
          beta, sigma, b_2tpl)
results_b3 = opt.root(Eul_err2, b3_init, args = (b3_args)).x

b2_args = (n_vec[0:], r_path[0:], w_path[0:],
          beta, sigma, results_b3[0])
results_b2 = opt.root(Eul_err1, b2_init, args = (b2_args))
results_b2

In [ ]:
import numpy as np
import scipy.optimize as opt

def Eul_err2(b3, *args):
    npath, rpath,  wpath, beta, sigma, b_2tpl = args
    c2 = (1 + rpath[0]) * b_2tpl + wpath[0] * npath[0] - b3
    c3 = (1 + rpath[1]) * b3 + wpath[1] * npath[1]
    MU2 = c2 ** (-sigma)
    MU3 = c3 ** (-sigma)
    eul_err = MU2 - beta * (1 + rpath[1]) * MU3
    
    return eul_err

def Eul_err1(b2, *args):
    npath, rpath,  wpath, beta, sigma, b3 = args
    c1 = wpath[0] * npath[0] - b2
    c2 = (1 + rpath[1]) * b2 + wpath[1] * npath[1] - b3
    MU1 = c1 ** (-sigma)
    MU2 = c2 ** (-sigma)
    eul_err = MU1 - beta * (1 + rpath[1]) * MU2
    return eul_err

def feasible(f_params, bvec_guess):
    nvec, A, alpha, delta = f_params
    b_2, b_3 = bvec_guess
    
    beta = 0.442
    sigma = 3
    b_2tpl = b_2
    
    b3_init = b_3
    b2_init = 0
    
    r1 = alpha * A * (sum(nvec[:2])/(b_2 + b_3))**(1-alpha) - delta
    r2 = alpha * A* (sum(nvec[1:2])/b_3)**(1-alpha) - delta
    w1 = (1-alpha) * A * (b_2 + b_3)/sum(nvec[:2])**alpha
    w2 = (1-alpha) * A * (b_3)/sum(nvec[1:2])**alpha
    w3_path = (w1, w2)
    r3_path = (r1, r2)
    
    b3_args = (nvec[1:], r3_path, w3_path, beta, sigma, b_2tpl)
    b3 = opt.root(Eul_err2, b3_init, args = (b3_args)).x
    
    r1 = alpha * A * (sum(nvec[:2])/(b_2 + b3[0]))**(1-alpha) - delta
    w0 = (1-alpha) * A * (b_2 + b3[0])/sum(nvec[:2])**alpha
    w2_path = (w0, w1)
    r2_path = (r1, r2)
    b2_args = (n_vec[0:], r2_path[0:], w2_path[0:], beta, sigma, b3[0])
    b2_init = 0
    b2 = opt.root(Eul_err1, b2_init, args = (b2_args)).x
    
    c1 = w0 * nvec[0] - b2[0]
    c2 = (1 + r1) * b2[0] + w1 * nvec[1] - b3[0]
    c3 = (1 + r2) * b3[0] + w2 * nvec[2]
     
    K_cnstr = b2[0] + b3[0] <= 0
    c_cnstr = c1 <= 0, c2 <= 0, c3 <= 0
    
    if c_cnstr[1] == True:
        b_cnstr = (c_cnstr[1], c_cnstr[1])
    else:
        b_cnstr = (c_cnstr[0], c_cnstr[2])
    
    return b_cnstr, c_cnstr, K_cnstr